# üé® SDXL Vintage Illustration Notebook ‚Äî PRO

Stable Diffusion XL / Turbo / SD 1.5 preconfigured for **vintage ink+watercolor** illustrations (old book style, retro cartoon aesthetic).

### Included
- **Mode Selector** (CPU / GPU basic / GPU optimal via `CONFIG`)
- **Gallery Manager (Flask web UI)** + **Logs** page
- **Text2Img** (default vintage style + custom scene prompt)
- **Img2Img** (photo ‚Üí vintage illustration)
- **ControlNet (Canny)** for pose/contour consistency
- **Upscale** (x4) for higher resolution
- **Color tone control** (dropdown presets + custom override)
- Saving images + JSON metadata to `/content/outputs` (timestamped)



In [ ]:
## ‚úÖ Tips

- Change **`color_tone`** or **`custom_tone`** any time, then regenerate.
- Use **`seed=None`** for randomization, or set a fixed integer for repeatability.
- For **Img2Img**, tweak **`strength`**:
  - lower = closer to original,
  - higher = stronger style.
- **ControlNet (Canny)** retains silhouette/contours. For different looks, adjust **`canny_low`/`canny_high`**.
- If memory errors occur, run **`free_memory()`** and re-run only the needed loader.


In [ ]:
# @title üöÄ Repo init
import os

repo_name = "sd-colab-gallery"
repo_url  = f"https://github.com/tekswirl25/{repo_name}.git"
repo_dir  = f"/content/{repo_name}"

if not os.path.exists(repo_dir):
    print("üì• Cloning repo...")
    !git clone {repo_url} {repo_dir}
else:
    print("üîÑ Repo exists, pulling latest changes...")
    %cd {repo_dir}
    !git pull

%cd {repo_dir}
print("‚úÖ Repo ready at", repo_dir)



In [ ]:
# @title üîß 1 Config init
import os, sys, pathlib, torch

# repo_dir –∑–∞–¥–∞—ë—Ç—Å—è –≤ üöÄ Repo init
sys.path.append(repo_dir)

# –í–∫–ª—é—á–∞–µ–º fallback –¥–ª—è macOS (–Ω–∞ Colab –Ω–µ –º–µ—à–∞–µ—Ç)
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

from scripts.logger import log_info
from scripts.config import VARIANTS, VARIANT_MODELS

CONFIG = {
    "MODE": "GPU_OPTIMAL",       # "CPU", "GPU_BASIC", "GPU_OPTIMAL"
    "MODEL_VARIANT": "SDXL",     # "SDXL", "TURBO", "SD15"
    "OUTPUT_DIR": "/content/outputs",
}
os.makedirs(CONFIG["OUTPUT_DIR"], exist_ok=True)

VARIANT = VARIANTS[CONFIG["MODEL_VARIANT"]]
DEFAULTS = VARIANT["defaults"]
AUTO_UPSCALE = VARIANT["auto_upscale"]

DEVICE = "cuda" if (VARIANT_MODELS.get("device","cuda")=="cuda" and torch.cuda.is_available()) else "cpu"
DTYPE  = torch.float16 if str(VARIANT_MODELS.get("dtype","fp16")).lower() in ("fp16","float16","half") else torch.float32

print("Torch version:", torch.__version__)
log_info(f"Config initialized: {CONFIG}, device={DEVICE}, dtype={DTYPE}")


In [ ]:
# @title üì¶ 2 Server (Flask gallery + logs)
from scripts.gallery_manager import start_gallery
start_gallery(CONFIG["OUTPUT_DIR"], port=8000, in_colab=True)




In [ ]:
# @title üì¶ 3 Install dependencies
!pip -q install diffusers==0.29.0 transformers accelerate xformers safetensors opencv-python-headless ipywidgets flask nest_asyncio

import os, torch
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
print("Dependencies installed. Torch version:", torch.__version__)




In [ ]:
# @title üîÅ 4 Imports & utils
from scripts.utils import (
    ts_now, base_name, save_image_and_meta,
    free_memory, list_images, canny_from_image
)
from scripts.logger import log_info, log_error
log_info(f"Utils loaded. OUTPUT_DIR={CONFIG['OUTPUT_DIR']}")



In [ ]:
# @title üß† 5 Model loaders
from scripts.loaders import (
    get_txt2img_pipe,
    get_img2img_pipe,
    get_controlnet_pipe,
    get_upscale_pipe,
    reset_pipes
)
log_info(f"Loader functions ready for variant: {CONFIG['MODEL_VARIANT']}")




In [ ]:
# @title üé® 6 Style base & Prompt builder
from scripts.prompt_builder import build_prompt

user_prompt = "A thoughtful man, slightly resembling Elon Musk."  #@param {type:"string"}
base_style  = "illustration"  #@param ["photoreal","illustration","anime"]
color_tone  = "vintage"       #@param ["warm","cool","vintage"]
negative    = ""              #@param {type:"string"}

final_prompt = build_prompt(user_prompt, style=base_style, tone=color_tone)
log_info(f"Prompt built. style={base_style}, tone={color_tone}")



In [ ]:
# @title üñº Text2Img
from PIL import Image

variant = CONFIG["MODEL_VARIANT"]
model_id = VARIANT_MODELS[variant]["txt2img"]

pipe = get_txt2img_pipe(model_id, DEVICE, DTYPE)

steps     = DEFAULTS["txt2img_steps"]
cfg_scale = DEFAULTS["txt2img_cfg"]
height, width = DEFAULTS["img_size"]

seed = 12345      #@param {type:"number"}
n    = 1          #@param {type:"number"}

saved = []
for _ in range(n):
    generator = torch.manual_seed(seed)
    out = pipe(
        prompt=final_prompt,
        negative_prompt=negative or None,
        height=height, width=width,
        guidance_scale=cfg_scale,
        num_inference_steps=steps,
        generator=generator,
    )
    im = out.images[0]
    meta = {
        "mode": "text2img",
        "prompt": final_prompt,
        "negative": negative,
        "steps": steps,
        "cfg_scale": cfg_scale,
        "size": [height, width],
        "seed": seed,
        "timestamp": ts_now(),
    }
    p, _ = save_image_and_meta(im, prefix="text2img", meta=meta, output_dir=CONFIG["OUTPUT_DIR"])
    saved.append(p)

log_info(f"Text2Img saved: {saved}")


In [ ]:
# @title üñº Img2Img
from PIL import Image

variant = CONFIG["MODEL_VARIANT"]
model_id = VARIANT_MODELS[variant]["img2img"]

src_path = ""    #@param {type:"string"}
strength = 0.6   #@param {type:"number"}
seed     = 12345 #@param {type:"number"}

if not src_path:
    raise ValueError("–ù—É–∂–Ω–æ —É–∫–∞–∑–∞—Ç—å –ø—É—Ç—å –∫ –∏—Å—Ö–æ–¥–Ω–æ–º—É –∏–∑–æ–±—Ä–∞–∂–µ–Ω–∏—é (src_path)")

image = Image.open(src_path).convert("RGB")
pipe_i2i = get_img2img_pipe(model_id, DEVICE, DTYPE)

generator = torch.manual_seed(seed)
out = pipe_i2i(
    prompt=final_prompt,
    negative_prompt=negative or None,
    image=image,
    strength=strength,
    generator=generator,
)

im = out.images[0]
meta = {
    "mode": "img2img",
    "prompt": final_prompt,
    "negative": negative,
    "strength": strength,
    "seed": seed,
    "timestamp": ts_now(),
}
p, _ = save_image_and_meta(im, prefix="img2img", meta=meta, output_dir=CONFIG["OUTPUT_DIR"])
log_info(f"Img2Img saved: {p}")


In [ ]:
# @title üß≠ 9 ControlNet (Canny)
from PIL import Image

variant = CONFIG["MODEL_VARIANT"]

if variant == "TURBO":
    base_model_id = VARIANT_MODELS["TURBO"]["controlnet_model"]
else:
    base_model_id = VARIANT_MODELS[variant]["txt2img"]

controlnet_id = VARIANT_MODELS[variant]["controlnet"]

pipe_canny = get_controlnet_pipe(base_model_id, controlnet_id, DEVICE, DTYPE)

control_path = ""   #@param {type:"string"}
low_thr = 100       #@param {type:"number"}
high_thr = 200      #@param {type:"number"}
seed     = 12345    #@param {type:"number"}

if not control_path:
    raise ValueError("–ù—É–∂–Ω–æ —É–∫–∞–∑–∞—Ç—å –ø—É—Ç—å –∫ –∏–∑–æ–±—Ä–∞–∂–µ–Ω–∏—é –¥–ª—è Canny (control_path)")

src = Image.open(control_path).convert("RGB")
canny_img = canny_from_image(src, low_thr, high_thr)

generator = torch.manual_seed(seed)
out = pipe_canny(
    prompt=final_prompt,
    negative_prompt=negative or None,
    image=canny_img,
    control_image=canny_img,
    num_inference_steps=DEFAULTS["controlnet_steps"],
    guidance_scale=DEFAULTS["controlnet_cfg"],
    generator=generator,
)

im = out.images[0]
meta = {
    "mode": "controlnet_canny",
    "prompt": final_prompt,
    "negative": negative,
    "low_thr": low_thr,
    "high_thr": high_thr,
    "seed": seed,
    "timestamp": ts_now(),
}
p, _ = save_image_and_meta(im, prefix="controlnet", meta=meta, output_dir=CONFIG["OUTPUT_DIR"])
log_info(f"ControlNet saved: {p}")


In [ ]:
# @title ‚¨ÜÔ∏è 10 Upscale x4
from PIL import Image

variant = CONFIG["MODEL_VARIANT"]
up_id   = VARIANT_MODELS[variant]["upscale"]

pipe_up = get_upscale_pipe(up_id, DEVICE, DTYPE)

in_path = ""   #@param {type:"string"}
seed    = 12345 #@param {type:"number"}

if not in_path:
    raise ValueError("–ù—É–∂–Ω–æ —É–∫–∞–∑–∞—Ç—å –ø—É—Ç—å –∫ –∏–∑–æ–±—Ä–∞–∂–µ–Ω–∏—é –¥–ª—è –∞–ø—Å–∫–µ–π–ª–∞ (in_path)")

img = Image.open(in_path).convert("RGB")
generator = torch.manual_seed(seed)

out = pipe_up(image=img, prompt=final_prompt, generator=generator)

im = out.images[0]
meta = {
    "mode": "upscale_x4",
    "prompt": final_prompt,
    "seed": seed,
    "timestamp": ts_now(),
}
p, _ = save_image_and_meta(im, prefix="upscale", meta=meta, output_dir=CONFIG["OUTPUT_DIR"])
log_info(f"Upscale saved: {p}")
